In this notebook, we resolve the problem
$$
\min_{u, f} \ \frac{1}{2}\int_{\Omega} |u-u_R|^2\textrm{d}x+\frac{\alpha}{2}\int_{\Omega} |f|^2\textrm{d}x
$$
for $\alpha\geq 0 $, subject to Poisson equation
$$
-\Delta u = f  \qquad \mathrm{in } \ \Omega=[0,1]^2
$$
with Dirichlet boundary conditions
$$
\begin{split}
u&= 0  \qquad \mathrm{on} \ \partial \Omega\\\end{split}
$$
where $u_R(x,y)=\dfrac{1}{2\pi^2 (1+4\alpha\pi^4)}\sin(\pi x)\sin(\pi y)$ and $f\in L^2 (\Omega)$.

We can prove that $f(x,y)=\dfrac{1}{1+4\alpha\pi^4}\sin(\pi x)\sin(\pi y)$ is the optimal control.

In [1]:
from __future__ import print_function
from dolfin import *
from dolfin_adjoint import *
import numpy as np
import scipy as sp
import cmath
import matplotlib.pyplot as plt
from mshr import *
import os
# Creating the mesh
n = 40
mesh = RectangleMesh(Point(0,0),Point(1,1),n,n,diagonal="left/right")
# Defining FEM spaces
V = FunctionSpace(mesh, "CG", 1)
W = V
# Defining state and control
f = Function(W, name='Control')
u = Function(V, name='State')
# Defining variational problem
v = TestFunction(V)
F = (inner(grad(u), grad(v)) - f*v)*dx
bc = DirichletBC(V, 0.0, "on_boundary")
solve(F == 0, u, bc)
# Creating reference solution
x = SpatialCoordinate(mesh)
w = Expression("sin(pi*x[0])*sin(pi*x[1])", degree=3) 
d = 1/(2*pi**2)
u_R = Expression("d*w", d=d, w=w, degree=3)
# Defining the functional
alpha = Constant(1e-6)
J = assemble((1/2*inner(u-u_R, u-u_R))*dx + alpha/2*f**2*dx)
control = Control(f)
rf = ReducedFunctional(J, control)
# and the optimization problem
f_opt = minimize(rf,tol=1.0e-10,options={"gtol": 1.0e-10})
# Defining the expressions of the analytical solution
f_analytic = Expression("1/(1+alpha*4*pow(pi, 4))*w", w=w, alpha=alpha, degree=6)
u_analytic = Expression("1/(2*pow(pi, 2))*f", f=f_analytic, degree=6)
# Computing the optimal states
f.assign(f_opt)
solve(F == 0, u, bc)
# and L2 errors
control_error = errornorm(f_analytic, f_opt)
state_error = errornorm(u_analytic, u)
print("h(min):           %e." % mesh.hmin())
print("Error in state:   %e." % state_error)
print("Error in control: %e." % control_error)
file = XDMFFile("f_opt.xdmf")
file.write_checkpoint(f_opt,'f_opt',0, XDMFFile.Encoding.HDF5, False)
file.close()
file = XDMFFile("u_opt.xdmf")
file.write_checkpoint(u,'u_opt',0, XDMFFile.Encoding.HDF5, False)
file.close()

h(min):           3.535534e-02.
Error in state:   1.844719e-05.
Error in control: 7.083576e-02.


In [2]:
import os
os.system('python3 Poisson.py')

0